# Lab 3

In [1]:
# Install required packages if needed
# !pip install SimpleITK torch torchio

## 1. Loading Images with SimpleITK

In [2]:
import SimpleITK as sitk
import numpy as np

# Load a 2D or 3D image (e.g., NIfTI, DICOM, etc.)
image_path = '../data/sample_image.nii.gz'  # Update with your image path
image = sitk.ReadImage(image_path)
print('Image size:', image.GetSize())
print('Image spacing:', image.GetSpacing())

ModuleNotFoundError: No module named 'SimpleITK'

## 2. Manipulating Imaging Arrays in NumPy and Torch

In [ ]:
import torch

# Convert SimpleITK image to NumPy array
image_np = sitk.GetArrayFromImage(image)  # shape: (z, y, x) for 3D, (y, x) for 2D
print('NumPy array shape:', image_np.shape)

# Manipulate with NumPy (e.g., normalize)
image_np_norm = (image_np - np.min(image_np)) / (np.max(image_np) - np.min(image_np))
print('Normalized min/max:', image_np_norm.min(), image_np_norm.max())

# Convert to Torch tensor
image_tensor = torch.from_numpy(image_np_norm).float()
print('Torch tensor shape:', image_tensor.shape)

## 3. Data Augmentation on 2D/3D Images using TorchIO

In [ ]:
import torchio as tio

# TorchIO expects images in (C, Z, Y, X) or (C, Y, X) format
if image_tensor.ndim == 3:
    image_tensor = image_tensor.unsqueeze(0)  # Add channel dimension
elif image_tensor.ndim == 2:
    image_tensor = image_tensor.unsqueeze(0).unsqueeze(0)

# Create a TorchIO Subject
subject = tio.Subject(
    image=tio.ScalarImage(tensor=image_tensor)
)

# Define a data augmentation pipeline
transform = tio.Compose([
    tio.RandomFlip(axes=(0, 1)),
    tio.RandomAffine(scales=(0.9, 1.1), degrees=10),
    tio.RandomNoise(mean=0, std=0.1),
    tio.RandomBiasField(coefficients=0.5)
])

# Apply augmentation
augmented = transform(subject)
aug_image = augmented.image.data
print('Augmented image shape:', aug_image.shape)

## 4. Visualize Original and Augmented Images (2D slice example)

In [ ]:
import matplotlib.pyplot as plt

# Show a middle slice for 3D, or the image for 2D
def show_slice(tensor, title):
    arr = tensor.squeeze().cpu().numpy()
    if arr.ndim == 3:
        idx = arr.shape[0] // 2
        arr = arr[idx]
    plt.imshow(arr, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

show_slice(image_tensor, 'Original Image')
show_slice(aug_image, 'Augmented Image')